In [ ]:
from poi.dataset.llm import load_prompt_completion_llm_dataset, extract_user_sids, find_all_user_sids_in_dataset, filter_test
from poi import settings

ds_dir = settings.DATASETS_DIR / "NYC" / "LLM Dataset" / "paper-nyc-base"

train_ds_path = ds_dir / "train_codebook.json"
test_ds_path = ds_dir / "test_codebook.json"

train_ds = load_prompt_completion_llm_dataset(train_ds_path)
test_ds = load_prompt_completion_llm_dataset(test_ds_path)



/home/vislab/poi/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
prompt = train_ds[0]["prompt"]
completion = train_ds[0]["completion"]

print(prompt)

user, sids = extract_user_sids(train_ds[0])
print(user)
print(sids)


Here is a record of a user's POI accesses, your task is based on the history to predict the POI that the user is likely to access at the specified time.
User_2 visited: <a_26><b_18><c_11><d_1> at 2012-04-20 07:23, <a_31><b_12><c_4> at 2012-04-20 21:32, <a_29><b_13><c_6> at 2012-04-20 21:34, <a_31><b_2><c_27> at 2012-04-21 13:20, <a_0><b_9><c_1> at 2012-04-21 20:45, <a_29><b_13><c_6> at 2012-04-21 22:29, <a_26><b_18><c_10><d_0> at 2012-04-23 06:23, <a_26><b_3><c_8> at 2012-04-23 06:23, <a_26><b_18><c_10><d_1> at 2012-04-23 06:23, <a_26><b_24><c_26> at 2012-04-23 07:16, <a_26><b_18><c_11><d_0> at 2012-04-23 07:17, <a_26><b_18><c_11><d_1> at 2012-04-23 07:17, <a_6><b_0><c_29> at 2012-04-23 07:17, <a_31><b_12><c_4> at 2012-04-23 10:55, <a_26><b_24><c_26> at 2012-04-24 06:59, <a_26><b_18><c_10><d_0> at 2012-04-24 06:59, <a_26><b_18><c_10><d_1> at 2012-04-24 06:59, <a_26><b_3><c_8> at 2012-04-24 06:59, <a_26><b_18><c_11><d_0> at 2012-04-24 07:57, <a_6><b_0><c_29> at 2012-04-24 07:57, <a_26><

In [ ]:
train_users, train_sids = find_all_user_sids_in_dataset(train_ds)
test_users, test_sids = find_all_user_sids_in_dataset(test_ds)

print("Users in test but not in train:", test_users - train_users)
print()

print("SIDs in test but not in train:", test_sids - train_sids)
print()


Users in test but not in train: {'User_1067', 'User_345', 'User_125', 'User_826', 'User_371', 'User_837', 'User_170'}

SIDs in test but not in train: {'<a_31><b_25><c_8><d_4>', '<a_9><b_20><c_29><d_3>', '<a_3><b_2><c_30>', '<a_6><b_4><c_25><d_1>', '<a_29><b_11><c_29>', '<a_19><b_0><c_7>', '<a_9><b_19><c_12>', '<a_17><b_17><c_12><d_3>', '<a_6><b_1><c_17><d_0>', '<a_27><b_7><c_7>', '<a_17><b_17><c_20><d_0>', '<a_28><b_17><c_8>', '<a_19><b_24><c_4><d_1>', '<a_28><b_24><c_7><d_2>', '<a_9><b_0><c_7><d_1>', '<a_24><b_7><c_3>', '<a_3><b_3><c_7><d_1>', '<a_17><b_27><c_23><d_0>', '<a_25><b_13><c_25>', '<a_26><b_6><c_13><d_1>', '<a_18><b_3><c_16><d_1>', '<a_28><b_24><c_14>', '<a_22><b_10><c_1>', '<a_28><b_24><c_7><d_1>', '<a_22><b_0><c_9><d_1>', '<a_6><b_25><c_31><d_2>', '<a_3><b_3><c_7><d_2>', '<a_3><b_29><c_7>', '<a_19><b_25><c_27>', '<a_4><b_6><c_17><d_0>', '<a_23><b_7><c_8>', '<a_7><b_17><c_26>', '<a_19><b_1><c_8><d_1>', '<a_26><b_30><c_14>', '<a_5><b_9><c_23><d_1>', '<a_27><b_31><c_13><d_1>

In [ ]:
filtered_test_ds = test_ds.filter(filter_test, fn_kwargs={"filter_users": test_users - train_users, "filter_sids": test_sids - train_sids})
filtered_users, filtered_sids = find_all_user_sids_in_dataset(filtered_test_ds)

print("Users in filtered test but not in train:", filtered_users - train_users)
print()
print("SIDs in filtered test but not in train:", filtered_sids - train_sids)
print()

print(len(filtered_test_ds) + "/", len(test_ds))


Users in filtered test but not in train: set()

SIDs in filtered test but not in train: set()



### Try to evaluate on filtered test dataset

In [ ]:
from poi.llm.evaluate import top_one_accuracy
from poi.llm import LLMConfig, load_fast_inference_model
from poi.dataset.llm import load_prompt_completion_llm_dataset, extract_user_sids, find_all_user_sids_in_dataset, filter_test
from poi import settings

ds_dir = settings.DATASETS_DIR / "NYC" / "LLM Dataset" / "paper-nyc-base"

train_ds_path = ds_dir / "train_codebook.json"
test_ds_path = ds_dir / "test_codebook.json"

train_ds = load_prompt_completion_llm_dataset(train_ds_path)
test_ds = load_prompt_completion_llm_dataset(test_ds_path)

train_users, train_sids = find_all_user_sids_in_dataset(train_ds)
test_users, test_sids = find_all_user_sids_in_dataset(test_ds)

test_ds = test_ds.filter(filter_test, fn_kwargs={"filter_users": test_users - train_users, "filter_sids": test_sids - train_sids})


config = LLMConfig(run_name="llama3-nyc-test")
model = load_fast_inference_model(config)

acc = top_one_accuracy(config, model, test_ds)
print(f"Top-1 accuracy: {acc}")


==((====))==  Unsloth 2025.10.9: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.516 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.10.9 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.
100%|██████████| 668/668 [04:08<00:00,  2.69it/s]

Top-1 accuracy: 0.3338323353293413


In [1]:
from poi.llm.evaluate import top_one_accuracy
from poi.llm import LLMConfig, load_fast_inference_model
from poi.dataset.llm import load_prompt_completion_llm_dataset, extract_user_sids, find_all_user_sids_in_dataset, filter_test
from poi import settings

ds_dir = settings.DATASETS_DIR / "NYC" / "LLM Dataset" / "ablation without SID"

train_ds_path = ds_dir / "train_id.json"
test_ds_path = ds_dir / "test_id.json"

train_ds = load_prompt_completion_llm_dataset(train_ds_path)
test_ds = load_prompt_completion_llm_dataset(test_ds_path)

train_users, train_sids = find_all_user_sids_in_dataset(train_ds)
test_users, test_sids = find_all_user_sids_in_dataset(test_ds)

test_ds = test_ds.filter(filter_test, fn_kwargs={"filter_users": test_users - train_users, "filter_sids": test_sids - train_sids})


config = LLMConfig(run_name="llama3-nyc-test-no-sid")
model = load_fast_inference_model(config)

acc = top_one_accuracy(config, model, test_ds, is_sid=False)
print(f"Top-1 accuracy: {acc}")


/home/vislab/poi/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Skipping import of cpp extensions due to incompatible torch version 2.8.0+cu128 for torchao version 0.14.0         Please see GitHub issue #2919 for more info


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.9: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.516 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.10.9 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.
100%|██████████| 793/793 [01:53<00:00,  6.99it/s]

Top-1 accuracy: 0.31273644388398486


In [ ]:
from poi.llm.evaluate import top_one_accuracy
from poi.llm import LLMConfig, load_fast_inference_model
from poi.dataset.llm import load_prompt_completion_llm_dataset, extract_user_sids, find_all_user_sids_in_dataset, filter_test
from poi import settings

ds_dir = settings.DATASETS_DIR / "NYC" / "LLM Dataset" / "paper-nyc-base"

train_ds_path = ds_dir / "train_codebook.json"
test_ds_path = ds_dir / "test_codebook.json"

train_ds = load_prompt_completion_llm_dataset(train_ds_path)
test_ds = load_prompt_completion_llm_dataset(test_ds_path)

train_users, train_sids = find_all_user_sids_in_dataset(train_ds)
test_users, test_sids = find_all_user_sids_in_dataset(test_ds)

test_ds = test_ds.filter(filter_test, fn_kwargs={"filter_users": test_users - train_users, "filter_sids": test_sids - train_sids})


config = LLMConfig(run_name="llama3-nyc-base")
model = load_fast_inference_model(config)

acc = top_one_accuracy(config, model, test_ds)
print(f"Top-1 accuracy: {acc}")


In [1]:
from poi.llm.evaluate import top_one_accuracy
from poi.llm import LLMConfig, load_fast_inference_model
from poi.dataset.llm import load_prompt_completion_llm_dataset, extract_user_sids, find_all_user_sids_in_dataset, filter_test
from poi import settings

ds_dir = settings.DATASETS_DIR / "NYC" / "LLM Dataset" / "ablation without Time"

train_ds_path = ds_dir / "train_codebook_notime.json"
test_ds_path = ds_dir / "test_codebook_notime.json"

train_ds = load_prompt_completion_llm_dataset(train_ds_path)
test_ds = load_prompt_completion_llm_dataset(test_ds_path)

train_users, train_sids = find_all_user_sids_in_dataset(train_ds)
test_users, test_sids = find_all_user_sids_in_dataset(test_ds)

test_ds = test_ds.filter(filter_test, fn_kwargs={"filter_users": test_users - train_users, "filter_sids": test_sids - train_sids})


/home/vislab/poi/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Skipping import of cpp extensions due to incompatible torch version 2.8.0+cu128 for torchao version 0.14.0         Please see GitHub issue #2919 for more info


🦥 Unsloth Zoo will now patch everything to make training faster!


IndexError: list index out of range

In [3]:
import re

def extract_user_sids(record: dict[str, str]) -> tuple[str, set[str]]:
    SID_PATTERN = r"(?:<\w+>)+"
    USER_PATTERN = r"user_\d+"

    prompt = record["prompt"].lower()
    completion = record["completion"].lower()

    user: str = re.findall(USER_PATTERN, prompt)[0]
    prompt_sids: list[str] = re.findall(SID_PATTERN, prompt)
    completion_sids: list[str] = re.findall(SID_PATTERN, completion)

    sids = set(prompt_sids) | set(completion_sids)
    return user, sids

for i in range(len(train_ds)):
    try:
        user, sids = extract_user_sids(train_ds[i])
    except Exception as e:
        print(e)
        print(train_ds[i])
    break
